1. Load thư viện

In [3]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import random
import os
from tqdm import tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
os.chdir("drive/MyDrive/Image")
print(os.getcwd())

/content/drive/MyDrive/Image


In [6]:
!ls

dataset  dataset.zip  driving_log.csv  IMG.zip


In [7]:
# !unzip dataset.zip -d Image

2. Load dữ liệu và tiền xử lý

In [8]:
# Lấy các đường dẫn ảnh
image_path = list(paths.list_images('dataset/'))

# Xáo trộn ngẫu nhiên các đường dẫn ảnh
random.shuffle(image_path)

In [9]:
# Format đường dẫn: dataset/tên_loài_hoa/tên_ảnh/
# Ví dụ: dataset/Bluebell/image_0241.jpg
labels = [p.split(os.path.sep)[-2] for p in image_path]
print(f"Label category: {labels[:20]}....")

# Chuyển đổi label từ caterogy sang số
trns = LabelEncoder()
labels = trns.fit_transform(labels)
print(f"Labels after encoding: {labels[:20]}....")

Label category: ['Tulip', 'Bluebell', 'ColtsFoot', 'Crocus', 'Pansy', 'Windflower', 'Bluebell', 'Windflower', 'Fritillary', 'Cowslip', 'ColtsFoot', 'Sunflower', 'Snowdrop', 'Bluebell', 'LilyValley', 'Pansy', 'Windflower', 'Fritillary', 'Daffodil', 'Fritillary']....
Labels after encoding: [15  0  2  4 11 16  0 16  8  3  2 13 12  0 10 11 16  8  5  8]....


3. Load Pre-trained Model

In [10]:
# Load pre-trained model VGG16
# weight = "imagenet": sử dụng weight đã train trên ImageNet dataset
# include_top = False: loại bỏ các lớp fully connected layers
model = VGG16(weights='imagenet', include_top=False)
model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None, None, 3)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, None, None, 64)      │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, None, None, 64)      │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, None, None, 64)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, None, None, 128)     │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, None, None, 128)     │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, None, None, 128)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, None, None, 256)     │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, None, None, 256)     │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, None, None, 256)     │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, None, None, 256)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, None, None, 512)     │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, None, None, 512)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, None, None, 512)     │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
list_image = []
with tqdm(total=len(image_path), unit='image', unit_scale=True, desc='Loading images') as pbar:
    for (_, path) in enumerate(image_path):
        image = load_img(path, target_size=(224, 224))
        image = img_to_array(image)

        image = np.expand_dims(image, axis=0)             # Mở rộng thêm 1 chiều (224, 224, 3) -> (1, 224, 224, 3)
        image = imagenet_utils.preprocess_input(image)    # Chuyển kênh màu RGB -> BGR và chuẩn hóa các giá trị pixels, mỗi pixels trừ đi [103.939, 116.779, 123.68]
        list_image.append(image)
        pbar.update(1)

list_image = np.vstack(list_image)  # Gộp một list thành mảng numpy lớn
print(list_image.shape)

Loading images: 100%|██████████| 1.36k/1.36k [05:50<00:00, 3.89image/s]


(1360, 224, 224, 3)


In [12]:
features = model.predict(list_image, verbose = 1)
print(features.shape)

features = features.reshape((features.shape[0], 7*7*512))

43/43 ━━━━━━━━━━━━━━━━━━━━ 828s 19s/step
(1360, 7, 7, 512)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, shuffle=True, test_size=0.2, random_state=42)

In [14]:
mlp = MLPClassifier(hidden_layer_sizes=(2048, ), activation='relu', solver='adam', alpha=0, max_iter=200, random_state=42, verbose=True)
mlp.fit(X_train, y_train)

mlp_preds = mlp.predict(X_test)
print(classification_report(y_test, mlp_preds))

Iteration 1, loss = 8.12904453
Iteration 2, loss = 1.50676775
Iteration 3, loss = 0.45719674
Iteration 4, loss = 0.10003140
Iteration 5, loss = 0.09852649
Iteration 6, loss = 0.01784312
Iteration 7, loss = 0.02007272
Iteration 8, loss = 0.00355429
Iteration 9, loss = 0.00148554
Iteration 10, loss = 0.00148866
Iteration 11, loss = 0.00149278
Iteration 12, loss = 0.00148844
Iteration 13, loss = 0.00148419
Iteration 14, loss = 0.00148255
Iteration 15, loss = 0.00148121
Iteration 16, loss = 0.00148001
Iteration 17, loss = 0.00147898
Iteration 18, loss = 0.00147818
Iteration 19, loss = 0.00147742
Iteration 20, loss = 0.00147674
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
              precision    recall  f1-score   support

           0       1.00      0.82      0.90        17
           1       0.75      0.82      0.78        11
           2       0.68      0.94      0.79        16
           3       0.69      0.50      0.58        18
        

In [15]:
log_reg = LogisticRegression(C = 0.1, solver='lbfgs', max_iter=200, random_state=42, verbose=True)
log_reg.fit(X_train, y_train)

logreg_preds = log_reg.predict(X_test)
print(classification_report(y_test, logreg_preds))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        17
           1       0.82      0.82      0.82        11
           2       1.00      0.69      0.81        16
           3       0.64      0.78      0.70        18
           4       0.79      1.00      0.88        11
           5       0.72      0.81      0.76        16
           6       1.00      0.90      0.95        20
           7       0.89      1.00      0.94        17
           8       1.00      1.00      1.00        13
           9       0.94      0.88      0.91        17
          10       0.85      0.85      0.85        20
          11       1.00      0.94      0.97        16
          12       0.79      0.85      0.81        13
          13       1.00      1.00      1.00        24
          14       0.93      0.93      0.93        14
          15       1.00      0.67      0.80         9
          16       1.00      0.95      0.97        20

    accuracy              